In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
from otc.metrics.metrics import effective_spread

import wandb

from tqdm.auto import tqdm

from scipy.stats import wilcoxon
import warnings


/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set here globally
exchange = "cboe"
models = ["classical"] # ["gbm", "classical"]
subset = "all" #"test"  # "test"
strategy = "supervised"  # "transfer"


In [3]:
key = f"{exchange}_all_{strategy}_{subset}"
dataset = f"fbv/thesis/{exchange}_{strategy}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(dataset)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in models:
    results = f"fbv/thesis/{exchange}_{model}_{strategy}_{subset}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact cboe_supervised_raw:latest, 1937.19MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact cboe_classical_supervised_all:latest, 1059.18MB. 1 files... 
/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
wandb:   1 of 1 files downloaded.  
Done. 0:0:18.9


In [4]:
# p. 35-38
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif subset == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=columns
    )


results = []
for i, model in tqdm(enumerate(models)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=models)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


1it [00:09,  9.18s/it]


In [5]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        #("gbm", "gbm(classical-size)"),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex)",
        ),
    ]
]


In [6]:
results_data


classical                                    \
          tick(ex) quote(ex) lr(ex) emo(ex) clnv(ex)   
index                                                  
799115        -1.0      -1.0   -1.0    -1.0     -1.0   
808408        -1.0       1.0    1.0     1.0      1.0   
811820        -1.0      -1.0   -1.0    -1.0     -1.0   
813492         1.0      -1.0    1.0     1.0      1.0   
812175         1.0      -1.0    1.0     1.0      1.0   
...            ...       ...    ...     ...      ...   
37155409       1.0      -1.0   -1.0    -1.0     -1.0   
37155410       1.0       1.0    1.0     1.0      1.0   
37104659      -1.0       1.0    1.0     1.0      1.0   
37108365      -1.0      -1.0   -1.0    -1.0     -1.0   
37155411      -1.0      -1.0   -1.0    -1.0     -1.0   

                                                                                       
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex)  
index                                                                                  
799115                                                  1.0                            
808408                                                  1.0                            
811820                                                  1.0                            
813492                                                 -1.0                            
812175                                                  1.0                            
...                                                     ...                            
37155409                                               -1.0                            
37155410                                                1.0                            
37104659                                               -1.0                            
37108365                                                1.0                            
37155411                                                1.0                            

[37155412 rows x 6 columns]

### Robustness Checks

In [7]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock options",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "myn <= 0.7",
    "myn (0.7-0.9]",
    "myn (0.9-1.1]",
    "myn (1.1-1.3]",
    "myn > 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)
half_spread = np.where(ask >= bid, (ask - bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        ((mid - half_spread) < trade_price) & (trade_price < (mid + half_spread)),
        1,  # inside
        np.where(
            (trade_price == (mid - half_spread)) | ((mid + half_spread) == trade_price),
            2,  # at quotes
            np.where(
                (trade_price < (mid - half_spread))
                | ((mid + half_spread) > trade_price),
                3,
                4,
            ),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [8]:
X_print.head(20)


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid
index,,,,,,,,,,
799115,1,1.10,C,Others,"(1,3]",2011,ttm (2-3] month,NaN,unknown,NaN
808408,1,2.80,C,Stock options,>11,2011,ttm <= 1 month,NaN,unknown,NaN
811820,1,6.85,P,Others,"(1,3]",2011,ttm <= 1 month,myn > 1.3,unknown,NaN
813492,-1,0.95,C,Others,"(0,1]",2011,ttm <= 1 month,myn (0.9-1.1],unknown,NaN
812175,1,2.15,C,Stock options,"(1,3]",2011,ttm <= 1 month,NaN,unknown,NaN
258163,-1,0.10,C,Others,>11,2011,ttm <= 1 month,NaN,unknown,NaN
813332,-1,1.05,C,Others,>11,2011,ttm (2-3] month,myn (0.9-1.1],unknown,NaN
806661,-1,0.28,C,Stock options,>11,2011,ttm (3-6] month,myn (0.7-0.9],unknown,NaN
808893,1,2.66,P,Index option,"(5,11]",2011,ttm (1-2] month,myn (0.9-1.1],unknown,NaN


In [9]:
X_print = pd.concat([X_print, results_data], axis=1)


In [10]:
X_print.head()


,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,"(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex))"
index,,,,,,,,,,,,,,,,
799115,1,1.10,C,Others,"(1,3]",2011,ttm (2-3] month,NaN,unknown,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
808408,1,2.80,C,Stock options,>11,2011,ttm <= 1 month,NaN,unknown,NaN,-1.0,1.0,1.0,1.0,1.0,1.0
811820,1,6.85,P,Others,"(1,3]",2011,ttm <= 1 month,myn > 1.3,unknown,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
813492,-1,0.95,C,Others,"(0,1]",2011,ttm <= 1 month,myn (0.9-1.1],unknown,NaN,1.0,-1.0,1.0,1.0,1.0,-1.0
812175,1,2.15,C,Stock options,"(1,3]",2011,ttm <= 1 month,NaN,unknown,NaN,1.0,-1.0,1.0,1.0,1.0,1.0


## Results Set Generation

In [11]:
LUT = {
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    # "(": "(",  # put interval start in math env
    # "]": "]",  # put interval end in math env
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [12]:
def set_tex_style(styler, caption, label):
    res = (
        styler.set_caption(caption)
        # .hide(axis="index")
        .apply(
            highlight_max, props="font-weight:bold;", axis=0
        )  # optionaly set subset....
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        # .hide(axis="index", level=0)
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [13]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


In [14]:
results_data.columns.tolist()


[('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex)')]

In [15]:
X_print.head()

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,"(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex))"
index,,,,,,,,,,,,,,,,
799115,1,1.10,C,Others,"(1,3]",2011,ttm (2-3] month,NaN,unknown,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
808408,1,2.80,C,Stock options,>11,2011,ttm <= 1 month,NaN,unknown,NaN,-1.0,1.0,1.0,1.0,1.0,1.0
811820,1,6.85,P,Others,"(1,3]",2011,ttm <= 1 month,myn > 1.3,unknown,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
813492,-1,0.95,C,Others,"(0,1]",2011,ttm <= 1 month,myn (0.9-1.1],unknown,NaN,1.0,-1.0,1.0,1.0,1.0,-1.0
812175,1,2.15,C,Stock options,"(1,3]",2011,ttm <= 1 month,NaN,unknown,NaN,1.0,-1.0,1.0,1.0,1.0,1.0


In [16]:
X_print[X_print["prox_q_binned"]=="unknown"]

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,prox_q_binned,mid,"(classical, tick(ex))","(classical, quote(ex))","(classical, lr(ex))","(classical, emo(ex))","(classical, clnv(ex))","(classical, trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex))"
index,,,,,,,,,,,,,,,,
799115,1,1.10,C,Others,"(1,3]",2011,ttm (2-3] month,NaN,unknown,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
808408,1,2.80,C,Stock options,>11,2011,ttm <= 1 month,NaN,unknown,NaN,-1.0,1.0,1.0,1.0,1.0,1.0
811820,1,6.85,P,Others,"(1,3]",2011,ttm <= 1 month,myn > 1.3,unknown,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,1.0
813492,-1,0.95,C,Others,"(0,1]",2011,ttm <= 1 month,myn (0.9-1.1],unknown,NaN,1.0,-1.0,1.0,1.0,1.0,-1.0
812175,1,2.15,C,Stock options,"(1,3]",2011,ttm <= 1 month,NaN,unknown,NaN,1.0,-1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37151178,-1,0.40,C,Stock options,"(0,1]",2017,ttm (3-6] month,myn (0.9-1.1],unknown,0.380000,-1.0,1.0,1.0,1.0,1.0,1.0
37152504,1,24.42,P,Others,"(0,1]",2017,ttm (6-12] month,myn (0.9-1.1],unknown,24.264999,-1.0,1.0,1.0,1.0,1.0,1.0
37155404,-1,0.80,C,Index option,>11,2017,ttm (3-6] month,myn <= 0.7,unknown,0.775000,-1.0,1.0,1.0,1.0,1.0,1.0


## Accuracy Calculation

In [17]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        acc_tot = accuracy_score(
            X_print["buy_sell"].astype("int8"), X_print[classifier]
        )

        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 7/7 [07:55<00:00, 67.95s/it]


In [18]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [19]:
master


classical                        \
                                     tick(ex)  quote(ex)     lr(ex)   
Option Type       C                 48.333239  62.872189  62.576298   
                  P                 49.271270  61.933208  61.705392   
                  all               48.747808  62.457200  62.191395   
Security Type     Index option      47.996949  52.496725  52.294268   
                  Others            48.162598  64.503744  64.098810   
                  Stock options     49.049448  62.654934  62.436720   
                  all               48.747808  62.457200  62.191395   
Trade Size        (0,1]             47.706219  56.951694  56.716233   
                  (1,3]             48.689388  62.308466  62.074939   
                  (3,5]             48.476472  63.331247  63.068187   
                  (5,11]            48.743513  63.810673  63.499790   
                  >11               50.153624  66.963248  66.671665   
                  all               48.747808  62.457200  62.191395   
Year              2011              46.719151  53.171360  52.941907   
                  2012              47.383208  59.894616  59.645826   
                  2013              49.537424  73.472891  73.225957   
                  2014              49.870446  63.405125  63.188717   
                  2015              49.117802  62.993987  62.688033   
                  2016              48.885331  62.853424  62.522685   
                  2017              49.127238  60.964208  60.696949   
                  all               48.747808  62.457200  62.191395   
Time to Maturity  ttm <= 1 month    48.538111  62.562456  62.206654   
                  ttm (1-2] month   49.258536  62.872765  62.727357   
                  ttm (2-3] month   48.581483  62.959274  62.738733   
                  ttm (3-6] month   48.925802  62.782891  62.587525   
                  ttm (6-12] month  48.914875  62.515530  62.353157   
                  ttm > 12 month    49.152126  58.752101  58.692313   
                  all               48.747808  62.457200  62.191395   
Moneyness         myn <= 0.7        48.617085  56.223652  55.965057   
                  myn (0.7-0.9]     48.507012  62.406112  62.105879   
                  myn (0.9-1.1]     49.067798  63.973627  63.706580   
                  myn (1.1-1.3]     47.305918  56.897141  56.713366   
                  myn > 1.3         47.324618  54.575629  54.389422   
                  all               48.747808  62.457200  62.191395   
Location to Quote at mid            45.231628  49.956272  47.157637   
                  inside            49.577239  71.207003  71.207003   
                  at quotes         47.813907  44.191998  44.191998   
                  outside           48.960874  55.298029  55.298029   
                  unknown           47.486979  40.587609  40.023136   
                  all               48.747808  62.457200  62.191395   

                                                          \
                                      emo(ex)   clnv(ex)   
Option Type       C                 49.481847  54.493577   
                  P                 48.692094  53.178256   
                  all               49.132810  53.912262   
Security Type     Index option      41.213969  42.827401   
                  Others            49.665599  54.643793   
                  Stock options     49.713470  54.728144   
                  all               49.132810  53.912262   
Trade Size        (0,1]             44.843982  48.899370   
                  (1,3]             48.889421  53.547230   
                  (3,5]             49.873272  54.628678   
                  (5,11]            49.774963  54.841650   
                  >11               53.091115  58.554321   
                  all               49.132810  53.912262   
Year              2011              41.627205  46.329584   
                  2012              47.683376  52.379077   
                  2013           

In [20]:
master.style.pipe(
    set_tex_style, caption=("master-short", "master-long"), label=f"{key}-master"
)


## Effective Spread 🚧

In [21]:
classifiers


[('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(ex)')]

In [22]:
eff_dfs = []


def stats(x, classifier):

    nom = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="nominal")
    rel = (
        effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="relative")
        * 100
    )

    # eff_spread_pred = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="none")
    # eff_spread_true = effective_spread(x["buy_sell"], x["TRADE_PRICE"], x["mid"], mode="none")
    # wilcoxon_res  = wilcoxon(eff_spread_pred, eff_spread_true, nan_policy="omit", zero_method="zsplit")

    return pd.Series(
        {
            "nominal": nom,
            "rel": rel,
            # 'statistic':wilcoxon_res.statistic,
            # 'pvalue':wilcoxon_res.pvalue
        }
    )


for criterion in tqdm(criterions):
    results = []

    for classifier in tqdm(classifiers):
        res = X_print.groupby([criterion])[
            ["TRADE_PRICE", "mid", classifier, "buy_sell"]
        ].apply(stats, classifier)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1, keys=classifiers).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{key.lower()}-{criterion.lower()}-eff-spread",
    )

    # store all result sets for later use
    eff_dfs.append(result_df)


100%|██████████| 7/7 [02:08<00:00, 18.37s/it]


In [23]:
master = pd.concat(eff_dfs, axis=1, keys=LUT_INDEX.values()).T


In [24]:
master


classical                           \
                                    tick(ex)                quote(ex)   
                                     nominal            rel   nominal   
Option Type       C                 0.017803   1.306965e+00  0.127183   
                  P                 0.015439   1.344986e+00  0.111718   
Security Type     Index option      0.044756  9.164154e+302  0.308482   
                  Others            0.013520   4.444471e-01  0.091192   
                  Stock options     0.015218   1.664563e+00  0.112734   
Trade Size        (0,1]             0.023666   1.212443e+00  0.146043   
                  (1,3]             0.016545   1.386538e+00  0.122504   
                  (3,5]             0.016202   1.436164e+00  0.114522   
                  (5,11]            0.011286  1.257254e+303  0.106451   
                  >11               0.014485  2.802996e+302  0.105878   
Year              2011              0.022716   7.674763e-01  0.141865   
                  2012              0.018378   2.356917e-01  0.109439   
                  2013              0.015062  -3.987065e-01  0.094467   
                  2014              0.014047  1.129726e+303  0.117139   
                  2015              0.021664  4.246183e+302  0.128574   
                  2016              0.013506   2.509930e+00  0.122155   
                  2017              0.014087   1.925721e+00  0.126984   
Time to Maturity  ttm <= 1 month    0.014392   1.359691e+00  0.101355   
                  ttm (1-2] month   0.015483   1.751178e+00  0.103096   
                  ttm (2-3] month   0.014176   1.190454e+00  0.118629   
                  ttm (3-6] month   0.015715  2.970433e+303  0.130034   
                  ttm (6-12] month  0.021196  2.297815e+303  0.154654   
                  ttm > 12 month    0.043189  2.304722e+303  0.274860   
Moneyness         myn <= 0.7        0.021270  -7.687329e+00  0.116567   
                  myn (0.7-0.9]     0.016262  8.809654e+302  0.091473   
                  myn (0.9-1.1]     0.014354   8.209424e-01  0.109517   
                  myn (1.1-1.3]     0.032402   6.182144e-01  0.214656   
                  myn > 1.3         0.033787   4.757286e-01  0.290681   
Location to Quote at mid            0.000000   0.000000e+00  0.000000   
                  inside            0.014853   2.625744e+00  0.111769   
                  at quotes         0.031601  -3.475254e+00  0.162045   
                  outside          -0.018247   1.333936e+01  0.358458   
                  unknown           0.044199            inf  0.199629   

                                                                            \
                                                     lr(ex)                  
                                              rel   nominal            rel   
Option Type       C                  1.339642e+01  0.127183   1.339642e+01   
                  P                  1.340340e+01  0.111718   1.340340e+01   
Security Type     Index option      9.164154e+302  0.308482  9.164154e+302   
                  Others             1.277710e+01  0.091192   1.277710e+01   
                  Stock options      1.367986e+01  0.112734   1.367986e+01   
Trade Size        (0,1]              1.282983e+01  0.146043   1.282983e+01   
                  (1,3]              1.212337e+01  0.122504   1.212337e+01   
                  (3,5]              1.299650e+01  0.114522   1.299650e+01   
                  (5,11]            1.257254e+303  0.106451  1.257254e+303   
                  >11               2.802996e+302  0.105878  2.802996e+302   
Year              2011               1.302708e+01  0.141865   1.302708e+01   
                  2012               1.350246e+01  0.109439   1.350246e+01   
                  2013               1.258078e+01  0.094467   1.258078e+01   
                  2014              1.129726e+303  0.117139  1.129726e+303   
                  2015              4.246183e+302  0.128574  4.246183e+302   
  

In [25]:
master.iloc[:, 0:20]


classical                           \
                                    tick(ex)                quote(ex)   
                                     nominal            rel   nominal   
Option Type       C                 0.017803   1.306965e+00  0.127183   
                  P                 0.015439   1.344986e+00  0.111718   
Security Type     Index option      0.044756  9.164154e+302  0.308482   
                  Others            0.013520   4.444471e-01  0.091192   
                  Stock options     0.015218   1.664563e+00  0.112734   
Trade Size        (0,1]             0.023666   1.212443e+00  0.146043   
                  (1,3]             0.016545   1.386538e+00  0.122504   
                  (3,5]             0.016202   1.436164e+00  0.114522   
                  (5,11]            0.011286  1.257254e+303  0.106451   
                  >11               0.014485  2.802996e+302  0.105878   
Year              2011              0.022716   7.674763e-01  0.141865   
                  2012              0.018378   2.356917e-01  0.109439   
                  2013              0.015062  -3.987065e-01  0.094467   
                  2014              0.014047  1.129726e+303  0.117139   
                  2015              0.021664  4.246183e+302  0.128574   
                  2016              0.013506   2.509930e+00  0.122155   
                  2017              0.014087   1.925721e+00  0.126984   
Time to Maturity  ttm <= 1 month    0.014392   1.359691e+00  0.101355   
                  ttm (1-2] month   0.015483   1.751178e+00  0.103096   
                  ttm (2-3] month   0.014176   1.190454e+00  0.118629   
                  ttm (3-6] month   0.015715  2.970433e+303  0.130034   
                  ttm (6-12] month  0.021196  2.297815e+303  0.154654   
                  ttm > 12 month    0.043189  2.304722e+303  0.274860   
Moneyness         myn <= 0.7        0.021270  -7.687329e+00  0.116567   
                  myn (0.7-0.9]     0.016262  8.809654e+302  0.091473   
                  myn (0.9-1.1]     0.014354   8.209424e-01  0.109517   
                  myn (1.1-1.3]     0.032402   6.182144e-01  0.214656   
                  myn > 1.3         0.033787   4.757286e-01  0.290681   
Location to Quote at mid            0.000000   0.000000e+00  0.000000   
                  inside            0.014853   2.625744e+00  0.111769   
                  at quotes         0.031601  -3.475254e+00  0.162045   
                  outside          -0.018247   1.333936e+01  0.358458   
                  unknown           0.044199            inf  0.199629   

                                                                            \
                                                     lr(ex)                  
                                              rel   nominal            rel   
Option Type       C                  1.339642e+01  0.127183   1.339642e+01   
                  P                  1.340340e+01  0.111718   1.340340e+01   
Security Type     Index option      9.164154e+302  0.308482  9.164154e+302   
                  Others             1.277710e+01  0.091192   1.277710e+01   
                  Stock options      1.367986e+01  0.112734   1.367986e+01   
Trade Size        (0,1]              1.282983e+01  0.146043   1.282983e+01   
                  (1,3]              1.212337e+01  0.122504   1.212337e+01   
                  (3,5]              1.299650e+01  0.114522   1.299650e+01   
                  (5,11]            1.257254e+303  0.106451  1.257254e+303   
                  >11               2.802996e+302  0.105878  2.802996e+302   
Year              2011               1.302708e+01  0.141865   1.302708e+01   
                  2012               1.350246e+01  0.109439   1.350246e+01   
                  2013               1.258078e+01  0.094467   1.258078e+01   
                  2014              1.129726e+303  0.117139  1.129726e+303   
                  2015              4.246183e+302  0.128574  4.246183e+302   
  

In [26]:
master.style.pipe(
    set_tex_style,
    caption=("master-short", "master-long"),
    label=f"{key}-master-eff-spread",
)


## Change in Parenthesis 🅾️

```latex
# https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290
\begin{table}
    \centering
    \caption{test of combination with change}
    \label{tab:combo}
    \begin{tabular}{lSSSSSSSS}
        \toprule
        {} & \multicolumn{2}{l}{Index option} & \multicolumn{2}{l}{Others} & \multicolumn{2}{l}{Stock options} & \multicolumn{2}{l}{all} \\
        \midrule
        classical-size & 1.0 & \parl-56.42\parr & 2.0 & \parl-74.35\parr & -73.5 & \parl-143.93\parr & 5.0 & \parl-67.33\parr \\
        \bottomrule
        \end{tabular}
\end{table}
```

In [27]:
foo = pd.DataFrame([[4, 3, 8, 5]])
bar = pd.DataFrame([[1.1, 2, 73, 5]], columns=foo.columns, index=foo.index)


In [28]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    for i, mul_col in enumerate(combo.columns):

        # define custom brackets that are not parsed by sunitx
        combo[[(mul_col[0], "pm")]] = (
            "\parl" + (combo[mul_col] - base[mul_col[0]]).round(2).astype(str) + "\parr"
        )
        # sort to group together columns
        combo.sort_index(axis=1, inplace=True)
    return combo


In [29]:
combo = combine_results(bar, foo)

# manually replace S with S[table-format=1.4(5)] if needed
combo.style.to_latex(
    f"combo.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label="tab:combo",
    caption="test of combination with change",
    multicol_align="l",
)


In [30]:
combo


0                    1                     2                    3  \
   nom              pm  nom              pm   nom              pm  nom   
0  1.1  \parl-2.9\parr  2.0  \parl-1.0\parr  73.0  \parl65.0\parr  5.0   

                  
              pm  
0  \parl0.0\parr